In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #choose whether to use gpu or cpu

In [4]:
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder'  #path to the folder containing the images
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\PastData\helical_size'
# first_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\first_half\first_quarter'     Bruno's computer
# second_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\first_half\second_quarter'
# third_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\second_half\third_quarter'
# fourth_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\second_half\fourth_quarter'
first_dir = r'C:\Users\Chappyyyyyy\Documents\size_folder\first_half\first_quarter' #chappy computer
second_dir = r'C:\Users\Chappyyyyyy\Documents\size_folder\first_half\second_quarter'
third_dir = r'C:\Users\Chappyyyyyy\Documents\size_folder\second_half\third_quarter'
fourth_dir = r'C:\Users\Chappyyyyyy\Documents\size_folder\second_half\fourth_quarter'


In [5]:
# multi-input dataset
class CustomImageDataset(Dataset):
    def __init__(self, first_dir, second_dir, third_dir, fourth_dir, transform=None):
        self.first_dir = first_dir
        self.second_dir = second_dir
        self.third_dir = third_dir
        self.fourth_dir = fourth_dir
        self.transform = transform
        # self.images1 = sorted(os.listdir(first_dir))
        self.images1 = sorted(os.listdir(first_dir), key=self.custom_sort_key)
        self.images2 = sorted(os.listdir(second_dir), key=self.custom_sort_key)
        self.images3 = sorted(os.listdir(third_dir), key=self.custom_sort_key)
        self.images4 = sorted(os.listdir(fourth_dir), key=self.custom_sort_key)
        self.labels1 = [self.extract_label(img) for img in self.images1]
        self.labels2 = [self.extract_label(img) for img in self.images2]
        self.labels3 = [self.extract_label(img) for img in self.images3]
        self.labels4 = [self.extract_label(img) for img in self.images4]
        
        #input verification
        assert len(self.images1) == len(self.images2) == len(self.images3) == len(self.images4)

        self.length = len(self.images1)

    def __len__(self):
        return self.length
        # return len(self.images1)

    def __getitem__(self, idx):
        img_name1 = os.path.join(self.first_dir, self.images1[idx])
        image1 = Image.open(img_name1)
        img_name2 = os.path.join(self.second_dir, self.images2[idx])
        image2 = Image.open(img_name2)
        img_name3 = os.path.join(self.third_dir, self.images3[idx])
        image3 = Image.open(img_name3)
        img_name4 = os.path.join(self.fourth_dir, self.images4[idx])
        image4 = Image.open(img_name4)
    
        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
            image3 = self.transform(image3)
            image4 = self.transform(image4)
        
        # print(self.labels1)
        # print(self.labels2)
        # print(self.labels3)
        # print(self.labels4)

        label1 = self.labels1[idx]
        label2 = self.labels2[idx]
        label3 = self.labels3[idx]
        label4 = self.labels4[idx]
        return image1, image2, image3, image4, label1, label2, label3, label4

    def custom_sort_key(self, item):
        # Extract numbers after 'Fig_' and 't-'
        fig_number = float(item.split('Fig_')[1].split('__')[0])
        t_number = float(item.split('t-')[1].split('_')[0])
        
        return fig_number, t_number
    
    def extract_label(self, img_name):
        # Assuming that the label is part of the filename before the first underscore
        label = float(img_name[-17:-5]) #this is the right code
        # label = str(img_name)
        return label

In [6]:
data_transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.45), (0.225))]) 

custom_dataset = CustomImageDataset(first_dir=first_dir, second_dir=second_dir, third_dir=third_dir, fourth_dir=fourth_dir, transform=data_transform)

# # Accessing the data
# for img, label in custom_dataset:
#     print(f"Image shape: {img.shape}, Label: {label}")

print(len(custom_dataset))

# train_set, val_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.15), int(len(custom_dataset)*0.100056)]) #splits data into training, validation and test sets
train_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.25013)])
print(len(train_set))
# print(len(val_set))
print(len(test_set))

3950
2962
988


In [7]:
#hyperparameters
num_epochs = 60
batch_size = 1
learning_rate = 0.0005

# LSTM Parameters
hidden_size = 100
input_size = 800
sequence_length = 4
num_layers = 2
output_size = 4




train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test = DataLoader(test_set, batch_size=batch_size, shuffle=True)
for (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in train:
    print(labels1.item(), labels2.item(), labels3.item(), labels4.item())
    break

24.62175 20.91379 4.577052 8.456822


In [8]:
class ConvNetLSTM(nn.Module): # note need to find out image size
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(ConvNetLSTM, self).__init__()
        self.conv1 = nn.Conv2d(1,8,10, padding='same') #in_channels, out_channels, kernel_size
        self.normalise1 = nn.BatchNorm2d(8)
        # self.pool1 = nn.AvgPool2d(10, stride=10)
        self.pool1 = nn.AvgPool2d(10, stride=10)
        self.conv2 = nn.Conv2d(8, 16, 10, padding='same')
        self.normalise2 = nn.BatchNorm2d(16)
        # self.pool2 = nn.AvgPool2d(2, stride=2)
        self.pool2 = nn.AvgPool2d(2, stride=2)
        self.pool3 = nn.AvgPool2d(1, stride=1)
        self.conv3 = nn.Conv2d(16, 32, 10, padding='same')
        self.normalise3 = nn.BatchNorm2d(32) 
        self.conv4 = nn.Conv2d(32, 32, 10, padding='same')
        self.fc0 = nn.Linear(32*5*5, 400)

        self.convcomb1 = nn.Conv2d(1, 16, 20, padding='same')
        self.convcomb2 = nn.Conv2d(16, 32, 20, padding='same')
        
        self.fc1 = nn.Linear(3200, 1600)
        self.fc2 = nn.Linear(1600,800)
        self.fc3 = nn.Linear(800,400)
        self.fc4 = nn.Linear(400,200)
        self.fc5 = nn.Linear(200,50)
        self.fc6 = nn.Linear(50,4)
        self.dropout = nn.Dropout(0.1)
        
        
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first = True,
        )
        # the size of the input to the lstm has to be (batch_size, seq, input_size)
        self.fclstm = nn.Linear(hidden_size, output_size)


    def forward(self, x, y, z, a):
        x = self.pool1(F.relu(self.normalise1(self.conv1(x)))) 
        x = self.pool2(F.relu(self.normalise2(self.conv2(x)))) 
        x = self.pool3(F.relu(self.normalise3(self.conv3(x))))
        x = self.pool3(F.relu(self.normalise3(self.conv4(x))))
        x = torch.flatten(x)  #flatten
        x = self.fc0(x)

        y = self.pool1(F.relu(self.normalise1(self.conv1(y)))) 
        y = self.pool2(F.relu(self.normalise2(self.conv2(y)))) 
        y = self.pool3(F.relu(self.normalise3(self.conv3(y))))
        y = self.pool3(F.relu(self.normalise3(self.conv4(y))))
        y = torch.flatten(y)  #flatten
        y = self.fc0(y)

        z = self.pool1(F.relu(self.normalise1(self.conv1(z))))
        z = self.pool2(F.relu(self.normalise2(self.conv2(z)))) 
        z = self.pool3(F.relu(self.normalise3(self.conv3(z))))
        z = self.pool3(F.relu(self.normalise3(self.conv4(z))))
        z = torch.flatten(z)  #flatten
        z = self.fc0(z)

        a = self.pool1(F.relu(self.normalise1(self.conv1(a)))) 
        a = self.pool2(F.relu(self.normalise2(self.conv2(a))))
        a = self.pool3(F.relu(self.normalise3(self.conv3(a))))
        a = self.pool3(F.relu(self.normalise3(self.conv4(a))))
        a = torch.flatten(a) #flatten
        a = self.fc0(a)


        # combined = torch.stack([x,y,z,a])
        # combined = combined.unsqueeze(0)
        # combined= combined.unsqueeze(0)
        # # print(combined.shape)
        # # print(combined)
        # combined = self.pool2(F.relu(self.normalise2(self.convcomb1(combined)))) 
        # # print(combined.shape)
        # # print(combined)
        # combined = self.pool2(F.relu(self.normalise3(self.convcomb2(combined)))) 
        # # combined = combined.view(-1, 32*5*5)  #flatten
        # combined = torch.flatten(combined)
        # # print(combined.shape)
        # combined = F.relu(self.fc1(combined))
        # combined = F.relu(self.fc2(combined))
        # combined = F.relu(self.fc3(combined))
        # combined = F.relu(self.fc4(combined))
        # combined = F.relu(self.fc5(combined))
        # combined = self.fc6(combined)
        # return combined
        combined = torch.stack([x,y,z,a])
        # resize the such that combined size is (batch_size, seq_length, input_size)
        combined = combined.unsqueeze(0)
        # begin LSTM
        h0 = torch.zeros(self.num_layers, combined.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, combined.size(0), self.hidden_size).to(device) 
        # forward propagate
        out, _ = self.lstm(combined, (h0,c0))  
        # decode hidden state of the final time step
        out = out[:,-1,:]
        out = out.fclstm(out)
        
        
      

        

In [9]:
model = ConvNetLSTM(input_size, hidden_size, num_layers, output_size).to(device)

# loss and optimizer
criterion = nn.MSELoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
#training loop
n_total_steps = len(train)
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in enumerate(train):
        images1 = images1.to(device)
        images2 = images2.to(device)
        images3 = images3.to(device)    
        images4 = images4.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        labels4 = labels4.to(device)

        #forward
        outputs = model(images1, images2, images3, images4)
        # print(labels)
        labels1 = labels1.float() 
        labels2 = labels2.float()
        labels3 = labels3.float()
        labels4 = labels4.float()
        labels = torch.cat((labels1, labels2, labels3, labels4), dim=0)
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # if (i+1) % 1000 ==0:
        # print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
        total_loss += loss.item()
    print(f'epoch {epoch+1}/{num_epochs}, average loss = {total_loss/len(train):.4f}')
            


c:\Users\Chappyyyyyy\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1009.)
  return F.conv2d(input, weight, bias, self.stride,


epoch 1/70, average loss = 53.4405
epoch 2/70, average loss = 37.1531
epoch 3/70, average loss = 28.1796
epoch 4/70, average loss = 24.9008
epoch 5/70, average loss = 21.6901
epoch 6/70, average loss = 17.9518
epoch 7/70, average loss = 15.9957
epoch 8/70, average loss = 14.5023
epoch 9/70, average loss = 13.7873
epoch 10/70, average loss = 12.9399
epoch 11/70, average loss = 12.3021
epoch 12/70, average loss = 11.5345
epoch 13/70, average loss = 11.2251
epoch 14/70, average loss = 10.5207
epoch 15/70, average loss = 10.5303
epoch 16/70, average loss = 9.3576
epoch 17/70, average loss = 8.1893
epoch 18/70, average loss = 8.0428
epoch 19/70, average loss = 7.6128
epoch 20/70, average loss = 7.2721
epoch 21/70, average loss = 7.0512
epoch 22/70, average loss = 6.5997
epoch 23/70, average loss = 6.4277
epoch 24/70, average loss = 6.3974
epoch 25/70, average loss = 6.3846
epoch 26/70, average loss = 5.8416
epoch 27/70, average loss = 6.0303
epoch 28/70, average loss = 6.0218
epoch 29/70, a

In [10]:
#Testing
with torch.no_grad(): # no need to calculate gradient
    squared_difference = 0
    for (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in test:
        images1 = images1.to(device)
        images2 = images2.to(device)
        images3 = images3.to(device)    
        images4 = images4.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        labels4 = labels4.to(device)

        #forward
        outputs = model(images1, images2, images3, images4)
    
        # print(predictions)
        squared_difference += (float(outputs[0]) - labels1) ** 2
        squared_difference += (float(outputs[1]) - labels2) ** 2
        squared_difference += (float(outputs[2]) - labels3) ** 2
        squared_difference += (float(outputs[3]) - labels4) ** 2
    
    rmse = torch.sqrt(squared_difference / (4*len(test)))
    print(f'RMSE = {rmse}')

RMSE = tensor([2.3533], dtype=torch.float64)


In [1]:
#save the model
# PATH = r'C:\Users\Chappyyyyyy\Desktop\PartIIB project 2023_2024\MultiInput\size_model.pth'
# torch.save(model.state_dict(), PATH)

NameError: name 'torch' is not defined